In [2]:
from llama_stack_client import LlamaStackClient, Agent
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os

namespace = "oom-test"
# Uncomment to run on a hosted Llama Stack server
client = LlamaStackClient(base_url="http://localhost:8321")

MODEL_ID = "llama-4-scout-17b-16e-w4a16"

base_agent_config = dict(
    model=MODEL_ID,
    instructions="You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    tools=[
        {
            "name": "mcp::openshift",
            "args": {"namespace": namespace}
        }
    ],
)

vanilla_agent_config = {
    **base_agent_config,
    "instructions": """
    You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.
    """,
}

vanilla_agent = Agent(client, **vanilla_agent_config)
prompt_chaining_session_id = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")

prompts = [
    """
    Analyze the namespace '{namespace}' by:
            1. Listing all pods 
            2. get pod logs for the pods listed above
            2. Checking events for any errors or warnings
            3. Providing initial assessment""",
    """Format the output as:
            - Namespace: {namespace}
            - Pod: [pod name]
            - Error Title: [error type/reason] 
            - Error Timestamp: [timestamp]
            - Error Description: [detailed error message]
            If no errors found, return: "No error found""",
]

for i, prompt in enumerate(prompts):    
    response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=prompt_chaining_session_id,
        stream=False,
    )
    print("************ Turn: ", i, "****************")
    print(response.output_message.content)
    print("\n")

************ Turn:  0 ****************
Your input is not sufficient. Please provide more details or specify the task you need help with.


************ Turn:  1 ****************
- Namespace: oom-test
- Pod: [oom-test-deployment-64cf479767-kx9b4, oom-test-deployment-64cf479767-rmccf]
- Error Title: [OOMKilled]
- Error Timestamp: [2025-10-08T14:41:55Z]
- Error Description: Container oom-test-container was terminated due to OOM
- Initial assessment: The pod oom-test-deployment-64cf479767-kx9b4 and oom-test-deployment-64cf479767-rmccf experienced OOM (Out of Memory) errors, resulting in the containers being terminated. This is likely due to the memory limit being too low for the workload. 

No error found in other namespaces.




In [3]:
from llama_stack_client import LlamaStackClient, Agent
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os

namespace = "oom-test"
# Uncomment to run on a hosted Llama Stack server
client = LlamaStackClient(base_url="http://localhost:8321")

MODEL_ID = "llama-4-scout-17b-16e-w4a16" #"llama-3-2-3b" #"r1-qwen-14b-w4a16" # "llama-4-scout-17b-16e-w4a16"

base_agent_config = dict(
    model=MODEL_ID,
    instructions="You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    tools=[
        {
            "name": "mcp::openshift",
            "args": {"namespace": namespace}
        }
    ],
)

vanilla_agent_config = {
    **base_agent_config,
    "instructions": """
    You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.
    """,
}

vanilla_agent = Agent(client, **vanilla_agent_config)
prompt_chaining_session_id = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")

prompts = [
    """
    Analyze the namespace '{namespace}' by:
            1. Listing all pods 
            2. get pod logs for the pods listed above
            2. Checking events for any errors or warnings
            3. Providing initial assessment""",
    """Format the output as JSON:
            {
                "namespace": "{namespace}",
                "analysis_results": [
                    {
                        "Pod": "[pod name]",
                        "Error Title": "[error type/reason]",
                        "Error Timestamp": "[timestamp]"
                        "Error Description": "[detailed error message]"
                    }
                ]
            }
            """,
]

for i, prompt in enumerate(prompts):    
    response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=prompt_chaining_session_id,
        stream=False,
    )
    print("************ Turn: ", i, "****************")
    print(response.output_message.content)
    print("\n")

************ Turn:  0 ****************
Your input is incomplete. Please provide further details or specify the task you need help with.


************ Turn:  1 ****************





In [3]:
# Create Jira Tickets from Error Analysis
print("🎫 Creating Jira Tickets from Error Analysis")
print("="*60)

import re
import json

# Extract error details from the previous analysis
def extract_error_details(analysis_text):
    """Extract error details from the analysis text"""
    errors = []
    
    # Look for JSON structure in the analysis
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', analysis_text, re.DOTALL)
    if json_match:
        try:
            json_data = json.loads(json_match.group(1))
            if 'analysis_results' in json_data:
                return json_data['analysis_results']
        except json.JSONDecodeError:
            pass
    
    # Fallback: look for structured error information
    lines = analysis_text.split('\n')
    current_error = {}
    
    for line in lines:
        line = line.strip()
        if line.startswith('- Pod:'):
            if current_error:
                errors.append(current_error)
            current_error = {'Pod': line.replace('- Pod:', '').strip()}
        elif line.startswith('- Error Title:'):
            current_error['Error Title'] = line.replace('- Error Title:', '').strip()
        elif line.startswith('- Error Timestamp:'):
            current_error['Error Timestamp'] = line.replace('- Error Timestamp:', '').strip()
        elif line.startswith('- Error Description:'):
            current_error['Error Description'] = line.replace('- Error Description:', '').strip()
    
    if current_error:
        errors.append(current_error)
    
    return errors

# Get the last response from the previous analysis
if 'response' in locals() and hasattr(response, 'output_message'):
    analysis_text = response.output_message.content
    print(f"📋 Extracted analysis text length: {len(analysis_text)} characters")
    
    # Extract error details
    error_details = extract_error_details(analysis_text)
    print(f"🔍 Found {len(error_details)} error(s) to create tickets for:")
    
    for i, error in enumerate(error_details):
        print(f"  {i+1}. Pod: {error.get('Pod', 'Unknown')}")
        print(f"     Error: {error.get('Error Title', 'Unknown')}")
        print(f"     Time: {error.get('Error Timestamp', 'Unknown')}")
        print()
    
    if error_details:
        # Create Agent for Jira ticket creation (using same pattern as previous cells)
        print("🤖 Setting up Agent for Jira ticket creation...")
        
        # Jira agent configuration (following the same pattern as vanilla_agent)
        jira_agent_config = {
            "model": MODEL_ID,
            "instructions": "You are an expert Jira administrator. Create Jira issues based on the provided error details from OpenShift analysis.",
            "sampling_params": {
                "strategy": {"type": "top_p", "temperature": 0.1, "top_p": 0.9},
            },
            "tools": [
                {
                    "name": "mcp::atlassian",
                    "args": {}
                }
            ],
        }
        
        # Create Jira agent
        jira_agent = Agent(client, **jira_agent_config)
        print("✅ Agent created for Jira ticket creation")
        
        # Create Jira session
        jira_session_id = jira_agent.create_session(session_name=f"jira_ticket_creation_{uuid.uuid4()}")
        print(f"✅ Jira session created: {jira_session_id}")
        
        # Create Jira tickets for each error
        for i, error in enumerate(error_details):
            print(f"\n🎫 Creating Jira ticket {i+1}/{len(error_details)}")
            print("-" * 40)
            
            # Create detailed Jira ticket prompt
            jira_prompt = f"""
            Create a Jira issue in the KAN project with the following details:
            
            Project: KAN
            Issue Type: Task
            Summary: OpenShift Error - {error.get('Error Title', 'Unknown Error')} in Pod {error.get('Pod', 'Unknown')}
            
            Description:
            Namespace: {namespace}
            Pod: {error.get('Pod', 'Unknown')}
            Error Type: {error.get('Error Title', 'Unknown')}
            Timestamp: {error.get('Error Timestamp', 'Unknown')}
            Error Details: {error.get('Error Description', 'No details available')}
            
            Priority: High (due to pod failure)
            Labels: openshift, pod-error, {error.get('Error Title', '').lower().replace(' ', '-')}
            
            Please create this Jira issue now.
            """
            
            try:
                # Create the Jira ticket using the same pattern as previous cells
                jira_response = jira_agent.create_turn(
                    messages=[
                        {
                            "role": "user",
                            "content": jira_prompt,
                        }
                    ],
                    session_id=jira_session_id,
                    stream=False,
                )
                
                print(f"✅ Jira ticket creation response:")
                print(f"   Response: {jira_response.output_message.content}")
                
            except Exception as e:
                print(f"❌ Failed to create Jira ticket {i+1}: {e}")
                import traceback
                traceback.print_exc()
        
        print(f"\n🎉 Completed creating Jira tickets for {len(error_details)} errors!")
        
    else:
        print("ℹ️  No errors found to create Jira tickets for.")
        
else:
    print("❌ No analysis response found. Please run the previous cells first.")


🎫 Creating Jira Tickets from Error Analysis
📋 Extracted analysis text length: 574 characters
🔍 Found 0 error(s) to create tickets for:
ℹ️  No errors found to create Jira tickets for.
